In [1]:
import pandas as pd
import geonamescache
import numpy as np

In [2]:
df = pd.read_csv("data/data_1.csv")

In [3]:
df.head(20)

,headline,countries,cities
0,Zika Outbreak Hits Miami,NaN,Miami
1,Could Zika Reach New York City?,NaN,New York City
2,First Case of Zika in Miami Beach,NaN,Miami Beach
3,"Mystery Virus Spreads in Recife, Brazil",Brazil,Recife
4,Dallas man comes down with case of Zika,NaN,Dallas
5,Trinidad confirms first Zika case,NaN,Trinidad
6,Zika Concerns are Spreading in Houston,NaN,Houston
7,Geneve Scientists Battle to Find Cure,NaN,Geneve
8,The CDC in Atlanta is Growing Worried,NaN,Atlanta
9,Zika Infested Monkeys in Sao Paulo,NaN,Sao Paulo


In [4]:
gc = geonamescache.GeonamesCache()

In [5]:
def f(x):
    return pd.Series([float('nan'),float('nan'),float('nan')])
df[["latitude","longitude","countrycode"]] = df.apply(f,axis=1)

In [6]:
import math
def f(x):
    city = x.cities
    country = x.countries
    k = gc.get_cities_by_name(city)
    result = pd.Series([float('nan'),float('nan'),float('nan')])
    if len(k)==0:
        pass
    elif len(k)==1:
        data = list(k[0].values())[0]
        result = pd.Series([data['latitude'],data['longitude'],data['countrycode']])

    elif len(k)>1:
        if type(country) == float and math.isnan(country):
            
            temp = []
            for j in k:
                data = list(j.values())[0]
                temp.append([data["population"],data['latitude'],data['longitude'],data["countrycode"]])
            result = pd.Series(sorted(temp,reverse=True)[0][1:])
        else:
            
            temp = []
            for j in k:
                data = list(j.values())[0]
                temp.append([data["population"],data['latitude'],data['longitude'],data["countrycode"]])
            found_country_code = gc.get_countries_by_names()[country]['iso']
            temp = sorted(temp,reverse=True)
            for selected in temp:
                if  found_country_code== selected[3]:
                    result = pd.Series(selected[1:])
                    break
    return result

df[["latitude","longitude","countrycode"]] = df.apply(f,axis=1)                 
          

df.head()

,headline,countries,cities,latitude,longitude,countrycode
0,Zika Outbreak Hits Miami,NaN,Miami,25.77427,-80.19366,US
1,Could Zika Reach New York City?,NaN,New York City,40.71427,-74.00597,US
2,First Case of Zika in Miami Beach,NaN,Miami Beach,25.79065,-80.13005,US
3,"Mystery Virus Spreads in Recife, Brazil",Brazil,Recife,-8.05389,-34.88111,BR
4,Dallas man comes down with case of Zika,NaN,Dallas,32.78306,-96.80667,US


In [9]:
df.drop("countries",axis=1,inplace=True)
df.rename(columns={"cities":"city"},inplace=True)

In [10]:
df.head()

,headline,city,latitude,longitude,countrycode
0,Zika Outbreak Hits Miami,Miami,25.77427,-80.19366,US
1,Could Zika Reach New York City?,New York City,40.71427,-74.00597,US
2,First Case of Zika in Miami Beach,Miami Beach,25.79065,-80.13005,US
3,"Mystery Virus Spreads in Recife, Brazil",Recife,-8.05389,-34.88111,BR
4,Dallas man comes down with case of Zika,Dallas,32.78306,-96.80667,US


In [11]:
df.to_csv("data/data_2.csv",index=False)